## Big Query 3 continued

In [ ]:
project = "cs544-spring2024" 
dataset = "lec_demo"

In [ ]:
# import statement
from google.cloud import bigquery

In [ ]:
# bigquery Client
bq = bigquery.Client()

In [ ]:
%load_ext google.cloud.bigquery

### Geopandas GeoDataFrame

- Installation requirements: `pip3 install geopandas`
- a geopandas GeoDataFrame is a subclass of pandas's DataFrame, and it has a geo column
- `to_dataframe` gives a pandas DataFrame
- `to_geodataframe` gives a geopandas GeoDataFrame

In [ ]:
# FIPS code for WI is 55
wi = bq.query("""
SELECT county_geom
FROM bigquery-public-data.geo_us_boundaries.counties
WHERE state_fips_code = '55'
""").to_dataframe()

In [ ]:
wi

In [ ]:
# FIPS code for WI is 55
wi = bq.query("""
SELECT county_geom
FROM bigquery-public-data.geo_us_boundaries.counties
WHERE state_fips_code = '55'
""").to_geodataframe()

In [ ]:
wi

In [ ]:
type(wi)

In [ ]:
wi.plot()

In [ ]:
wi.plot(color="0.9", edgecolor="0")

### Export WI to our private table

Why? So as to not keep querying the whole US data to retrieve just WI dataset.

In [ ]:
bq.query("""
CREATE OR REPLACE TABLE `cs544-spring2024.lec_demo.wi`
AS
SELECT county_name, county_geom
FROM bigquery-public-data.geo_us_boundaries.counties
WHERE state_fips_code = '55'
""").to_geodataframe()

In [ ]:
wi = bq.query("SELECT * FROM lec_demo.wi").to_geodataframe()
wi.head(3)

In [ ]:
with open("wi.geojson", "w") as f:
    f.write(wi.to_json())

### ST_\<SOME_FUNCTION\>

ST => Spacial Type

In [ ]:
bq.query("SELECT * FROM lec_demo.wi").to_geodataframe().plot(color="0.9", edgecolor="0")

### `ST_UNION_AGG`

- could be used with `GROUP BY`

In [ ]:
bq.query("SELECT ST_UNION_AGG(county_geom) FROM lec_demo.wi").to_geodataframe().plot(color="0.9", edgecolor="0")

### `ST_CENTROID` and `ST_CENTROID_AGG`

In [ ]:
ax = bq.query("SELECT * FROM lec_demo.wi").to_geodataframe().plot(color="0.9", edgecolor="0")
bq.query("SELECT ST_CENTROID(county_geom) FROM lec_demo.wi").to_geodataframe().plot(ax=ax, color="r")

In [ ]:
ax = bq.query("SELECT * FROM lec_demo.wi").to_geodataframe().plot(color="0.9", edgecolor="0")
bq.query("SELECT ST_CENTROID_AGG(county_geom) FROM lec_demo.wi").to_geodataframe().plot(ax=ax, color="r")

### `ST_GEOGPOINT`

In [ ]:
bq.query("""
SELECT *
FROM lec_demo.applications
""").to_dataframe()

In [ ]:
# doesn't work because we need geo column
# lat / lon are just floats
bq.query("""
SELECT *
FROM lec_demo.applications
""").to_geodataframe()

In [ ]:
bq.query("""
SELECT *, ST_GEOGPOINT(longitude, latitude) AS loc
FROM lec_demo.applications
""").to_geodataframe()

#### Creating a `houses` table

In [ ]:
bq.query("""
CREATE OR REPLACE TABLE lec_demo.houses
AS
SELECT *, ST_GEOGPOINT(longitude, latitude) AS loc
FROM lec_demo.applications
""").to_geodataframe()

In [ ]:
ax = bq.query("SELECT * FROM lec_demo.wi").to_geodataframe().plot(color="0.9", edgecolor="0")
bq.query("SELECT * FROM lec_demo.houses").to_geodataframe().plot(ax=ax)

In [ ]:
import geopandas
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
ax = world.plot(color="0.9", edgecolor="0")
bq.query("SELECT * FROM lec_demo.houses").to_geodataframe().plot(ax=ax, color="r")

#### What are the ten houses closest to the WI capitol?  
- `ST_DISTANCE`
- `ST_GEOGPOINT(-89.384107, 43.074715)`

In [ ]:
%%bigquery
SELECT *, ST_DISTANCE(loc, ST_GEOGPOINT(-89.384107, 43.074715)) / 1000 AS km
FROM lec_demo.houses
ORDER BY km ASC
LIMIT 10

In [ ]:
ax = bq.query("SELECT * FROM lec_demo.wi").to_geodataframe().plot(color="0.9", edgecolor="0")
bq.query("""
SELECT *, ST_DISTANCE(loc, ST_GEOGPOINT(-89.384107, 43.074715)) / 1000 AS km
FROM lec_demo.houses
ORDER BY km ASC
LIMIT 10
""").to_geodataframe().plot(ax=ax, color="r")

### Spatial JOIN

- `CROSS JOIN` followed by a filter using `ST_WITHIN` or `ST_CONTAINS`
- or `INNER JOIN` with the filter inside `ON` clause
  
#### What houses are in WI?

In [ ]:
%%bigquery
SELECT houses.*, wi.county_name
FROM lec_demo.houses CROSS JOIN lec_demo.wi
WHERE ST_Within(houses.loc, wi.county_geom)

In [ ]:
ax = bq.query("SELECT * FROM lec_demo.wi").to_geodataframe().plot(color="0.9", edgecolor="0")
bq.query("""
SELECT houses.*, wi.county_name
FROM lec_demo.houses CROSS JOIN lec_demo.wi
WHERE ST_Within(houses.loc, wi.county_geom)
""").to_geodataframe().plot(ax=ax, color="r")

### `ST_SNAPTOGRID`, `ST_GEOHASH`

- enables us to provide approximate location instead of actual location
- very useful to avoid privacy concerns

In [ ]:
ax = bq.query("SELECT * FROM lec_demo.wi").to_geodataframe().plot(color="0.9", edgecolor="0")

# exact locations:
bq.query("""
SELECT houses.*, wi.county_name
FROM lec_demo.houses CROSS JOIN lec_demo.wi
WHERE ST_Within(houses.loc, wi.county_geom)
""").to_geodataframe().plot(ax=ax, color="k")

# approx locations:
bq.query("""
SELECT ST_SNAPTOGRID(houses.loc, 0.1), wi.county_name
FROM lec_demo.houses CROSS JOIN lec_demo.wi
WHERE ST_Within(houses.loc, wi.county_geom)
""").to_geodataframe().plot(ax=ax, color="r")

In [ ]:
bq.query("""
SELECT houses.loc, ST_GEOHASH(houses.loc, 3), wi.county_name
FROM lec_demo.houses CROSS JOIN lec_demo.wi
WHERE ST_Within(houses.loc, wi.county_geom)
""").to_dataframe()

### Bigquery ML

In [ ]:
import pandas as pd
import numpy as np
df = pd.DataFrame({"x1": np.random.randint(0, 10, 100).astype(float), 
                   "x2": np.random.randint(0, 3, 100).astype(float)})
df["y"] = df["x1"] + df["x2"] + np.random.rand(len(df))
df.head()

In [ ]:
# gbq: google big query
df.to_gbq(f"{project}.{dataset}.sampleml", if_exists="replace")

### Train-test split using `rand()`

In [ ]:
%%bigquery
SELECT x1,x2,y,rand() < 0.25 AS test
FROM lec_demo.sampleml

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE lec_demo.traintest

AS

SELECT x1,x2,y,rand() < 0.25 AS test
FROM lec_demo.sampleml

### Creating and training a `LINEAR_REG` model

In [ ]:
%%bigquery 
select x1, x2, y
from lec_demo.traintest
where NOT test

In [ ]:
%%bigquery
CREATE OR REPLACE MODEL lec_demo.lr
OPTIONS(model_type="LINEAR_REG", INPUT_LABEL_COLS=["y"])

AS

select x1, x2, y
from lec_demo.traintest
where NOT test

### Model weights: `ML.WEIGHTS`

In [ ]:
%%bigquery
SELECT *
FROM ML.WEIGHTS(MODEL lec_demo.lr)

### Using the model to make predictions: `ML.PREDICT`

In [ ]:
%%bigquery
SELECT *
FROM ML.PREDICT(MODEL lec_demo.lr, (
    select x1, x2, y
    from lec_demo.traintest
    where test
))

In [ ]:
%%bigquery
SELECT *
FROM ML.PREDICT(MODEL lec_demo.lr, (
    select y, x1, x2
    from lec_demo.traintest
    where test
))

For `ML.PREDICT`, label column is optional.

In [ ]:
%%bigquery
SELECT *
FROM ML.PREDICT(MODEL lec_demo.lr, (
    select x1, x2
    from lec_demo.traintest
    where test
))

### Evaluating how well the model is performing: `ML.EVALUATE`

For `ML.EVALUATE`, label column is required.

In [ ]:
%%bigquery df
SELECT *
FROM ML.EVALUATE(MODEL lec_demo.lr, (
    select x1, x2
    from lec_demo.traintest
    where test
))

In [ ]:
%%bigquery df
SELECT *
FROM ML.EVALUATE(MODEL lec_demo.lr, (
    select x1, x2, y
    from lec_demo.traintest
    where test
))

In [ ]:
df

In [ ]:
df.at[0, "r2_score"]

**FINAL EXAM CUT OFF POINT**
***

### Transformers

In [ ]:
df = pd.DataFrame({"x1": np.random.randint(0, 10, 100).astype(float), 
                   "x2": np.random.randint(0, 3, 100).astype(float)})
df["y"] = 2 * df["x1"] ** 2 - 3 * df["x2"] + np.random.rand(len(df))
df.head()

In [ ]:
df.to_gbq(f"{project}.{dataset}.tbl1")

In [ ]:
%%bigquery
CREATE OR REPLACE MODEL lec_demo.m1
OPTIONS(model_type="LINEAR_REG", INPUT_LABEL_COLS=["y"])

AS

select x1, x2, y
from lec_demo.tbl1

In [ ]:
%%bigquery df
SELECT *
FROM ML.EVALUATE(MODEL lec_demo.m1, (
    select x1, x2, y
    from lec_demo.tbl1
))

In [ ]:
df

In [ ]:
%%bigquery
select x1, POWER(x1, 2) AS x1_2, x2, POWER(x2, 2) AS x2_2, y
from lec_demo.tbl1
LIMIT 5

In [ ]:
%%bigquery
CREATE OR REPLACE MODEL lec_demo.m2
TRANSFORM(x1, POWER(x1, 2) AS x1_2, x2, POWER(x2, 2) AS x2_2, y)
OPTIONS(model_type="LINEAR_REG", INPUT_LABEL_COLS=["y"])

AS

select x1, x2, y
from lec_demo.tbl1

In [ ]:
%%bigquery df
SELECT *
FROM ML.EVALUATE(MODEL lec_demo.m2, (
    select x1, x2, y
    from lec_demo.tbl1
))

In [ ]:
df

#### Compare m1 to m2

In [ ]:
%%bigquery
SELECT *
FROM ML.EVALUATE(MODEL lec_demo.m1, (
    select x1, x2, y
    from lec_demo.tbl1
))
UNION ALL
SELECT *
FROM ML.EVALUATE(MODEL lec_demo.m2, (
    select x1, x2, y
    from lec_demo.tbl1
))

In [ ]:
%%bigquery df
SELECT "m1" as model, r2_score
FROM ML.EVALUATE(MODEL lec_demo.m1, (
    select x1, x2, y
    from lec_demo.tbl1
))
UNION ALL
SELECT "m2" as model, r2_score
FROM ML.EVALUATE(MODEL lec_demo.m2, (
    select x1, x2, y
    from lec_demo.tbl1
))

In [ ]:
df

In [ ]:
ax = df.set_index("model").plot.bar(figsize=(3, 3))
ax.set_ylabel("R^2 Score")

### OneHot encoding example

In [ ]:
df = pd.DataFrame({"x1": np.random.choice(["A", "B", "C"], 100), 
                   "x2": np.random.randint(0, 10, 100).astype(float)})
df["y"] = df["x1"].map({"A":1, "B":5, "C":15}) + 2 * df["x2"] + np.random.rand(len(df))
df.head(3)

In [ ]:
df.to_gbq(f"{project}.{dataset}.tbl2")

In [ ]:
%%bigquery
CREATE MODEL lec_demo.m3
OPTIONS(model_type="LINEAR_REG", input_label_cols=["y"])

AS

SELECT x1, x2, y
FROM lec_demo.tbl2

In [ ]:
%%bigquery
SELECT "m3" as model, r2_score
FROM ML.EVALUATE(MODEL lec_demo.m3, (
    select x1, x2, y
    from lec_demo.tbl2
))

In [ ]:
%%bigquery df 
SELECT *
FROM ML.WEIGHTS(MODEL lec_demo.m3)

In [ ]:
df

In [ ]:
w = df.set_index("processed_input").at["x1", "category_weights"].tolist()
w

In [ ]:
weights = pd.DataFrame(w)
weights

In [ ]:
weights.set_index("category").plot.bar()